In [1]:
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv

In [4]:

class DatabaseConnection:
    def __init__(self):
        # AWS RDS configuration
        self.config = {
            'host': 'redditdb.cbkuy486ce24.ap-south-1.rds.amazonaws.com',
            'user': 'admin',     # Change this to your RDS master username (usually 'admin')
            'password': 'admin123',  # Replace with your actual RDS password
            'database': 'reddit01',
            'port': 3306
        }
        self.connection = None
        self.cursor = None

    def connect(self):
        """Establish connection to the database"""
        try:
            self.connection = mysql.connector.connect(**self.config)
            if self.connection.is_connected():
                db_info = self.connection.get_server_info()
                self.cursor = self.connection.cursor(dictionary=True)
                print(f"Successfully connected to MySQL database version {db_info}")
                return True
        except Error as e:
            print(f"Error connecting to MySQL Database: {e}")
            return False

    def disconnect(self):
        """Close database connection"""
        if self.connection and self.connection.is_connected():
            if self.cursor:
                self.cursor.close()
            self.connection.close()
            print("Database connection closed")

In [6]:
def fetch_reddit_data(db_connection, query, params=None):
    """Fetches data from the reddit_posts table.

    Args:
        db_connection: An instance of the DatabaseConnection class.
        query: The SQL query string.
        params: Optional parameters for the query (to prevent SQL injection).

    Returns:
        A list of dictionaries, where each dictionary represents a row.
        Returns None if there's an error.
    """
    try:
        cursor = db_connection.cursor
        cursor.execute(query, params)
        results = cursor.fetchall()
        return results
    except Error as e:
        print(f"Error fetching data: {e}")
        return None
    

In [2]:
db = DatabaseConnection()
connect = db.connect()
print(connect)

Successfully connected to MySQL database version 8.0.39
True


In [9]:
import pandas as pd

In [6]:
def import_scraped_data_to_db(db, scraped_data):
    try:
        # Convert the scraped data to DataFrame
        df = pd.DataFrame(scraped_data)
        
        # Convert created_utc to datetime if it's in unix timestamp format
        if 'created_utc' in df.columns:
            df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')
        
        # Prepare insert query
        insert_query = """
        INSERT INTO reddit_posts 
        (id, author, title, text, url, created_utc, score, num_comments, subreddit)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        # Convert DataFrame to list of tuples for batch insert
        values = df.fillna('').apply(tuple, axis=1).tolist()
        
        # Batch insert records
        batch_size = 1000
        for i in range(0, len(values), batch_size):
            batch = values[i:i + batch_size]
            try:
                db.cursor.executemany(insert_query, batch)
                db.connection.commit()
                print(f"Inserted records {i} to {i + len(batch)}")
            except Error as e:
                print(f"Error inserting batch: {e}")
                db.connection.rollback()
                
        print(f"Data import completed successfully! Total records inserted: {len(df)}")
        
    except Exception as e:
        print(f"Error during import: {e}")

# Example usage:
# Assuming your scraped_data is a list of dictionaries or similar structure
# import_scraped_data_to_db(db, scraped_data)

In [4]:
def create_reddit_table(db):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS reddit_posts (
        id VARCHAR(255) PRIMARY KEY,
        author VARCHAR(255) NOT NULL,
        title TEXT NOT NULL,
        text LONGTEXT,
        url TEXT,
        created_utc TIMESTAMP,
        score INT DEFAULT 0,
        num_comments INT DEFAULT 0,
        subreddit VARCHAR(255) NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
    );
    """
    
    try:
        db.cursor.execute(create_table_query)
        db.connection.commit()
        print("Table 'reddit_posts' created successfully!")
        
        # Create indexes for better query performance
        indexes = [
            "CREATE INDEX idx_author ON reddit_posts(author);",
            "CREATE INDEX idx_subreddit ON reddit_posts(subreddit);",
            "CREATE INDEX idx_created_utc ON reddit_posts(created_utc);",
            "CREATE INDEX idx_score ON reddit_posts(score);"
        ]
        
        for index_query in indexes:
            try:
                db.cursor.execute(index_query)
                db.connection.commit()
            except Error as e:
                # Skip if index already exists
                if e.errno != 1061:  # 1061 is MySQL error for duplicate index
                    print(f"Error creating index: {e}")
                
        print("Indexes created successfully!")
        
    except Error as e:
        print(f"Error creating table: {e}")



In [2]:
from src.reddit_scraper import RedditScraper
from src.settings import Settings


In [6]:
scraper = RedditScraper()
df = scraper.scrape_subreddit(
        subreddit_names=Settings.DEFAULT_SUBREDDITS,
        post_limit=Settings.DEFAULT_POST_LIMIT 
    )

2024-12-16 17:19:48,726 - RedditScraper - INFO - Attempting to create Reddit client
2024-12-16 17:19:48,730 - RedditScraper - INFO - Reddit client successfully created and verified
2024-12-16 17:19:53,952 - RedditScraper - INFO - Collected 10 posts from r/python
2024-12-16 17:19:57,601 - RedditScraper - INFO - Collected 10 posts from r/learnpython


AttributeError: 'DataFrame' object has no attribute 'rows'

In [10]:
import_scraped_data_to_db(db=db,scraped_data=df)

Inserted records 0 to 20
Data import completed successfully! Total records inserted: 20


In [7]:


db_conn = DatabaseConnection()
try:
    if db_conn.connect():
        limit = 20  # Set your desired limit

        query = "SELECT * FROM reddit_posts LIMIT %s;" # Limit in the query
        params = (limit,) # Limit passed as a parameter

        all_posts = fetch_reddit_data(db_conn, query, params)
        if all_posts:
            print(f"Retrieved {len(all_posts)} posts (limit was {limit}):")
            for post in all_posts:
                print(post)
        else:
             print("No posts found or an error occurred.") # Handle the case where no posts are found

finally:  # Ensure the connection is closed even if errors occur
    db_conn.disconnect()



Successfully connected to MySQL database version 8.0.39
Retrieved 20 posts (limit was 20):
{'id': '1heo2nz', 'author': 'Antique-Bowl-6384', 'title': 'Python comments', 'text': 'i found new comments:\n```\n"comment"\n2\n1.0\n#comment\n"""\ncomment\n"""\n["comment"]\n("comment")\n{"comment"}\n{"comment":"okay"}\n```\nif you dont use data types as variable you can use for comments like docstring', 'url': 'https://reddit.com/r/Python/comments/1heo2nz/python_comments/', 'created_utc': datetime.datetime(2024, 12, 15, 8, 7, 31), 'score': 0, 'num_comments': 7, 'subreddit': 'python', 'created_at': datetime.datetime(2024, 12, 16, 11, 52, 9), 'updated_at': datetime.datetime(2024, 12, 16, 11, 52, 9)}
{'id': '1heo8ps', 'author': 'Jaxondevs', 'title': "PyGyat, What is everyone's thoughts on it", 'text': "[https://github.com/shamith09/pygyat](https://github.com/shamith09/pygyat)\n\nI saw this today and was wondering about everyone's thoughts", 'url': 'https://reddit.com/r/Python/comments/1heo8ps/pygy

In [8]:
def get_posts_by_subreddit(db_connection, subreddit):
    """Fetches posts from a specific subreddit.

    Args:
        db_connection: An instance of the DatabaseConnection class.
        subreddit: The name of the subreddit.

    Returns:
        A list of dictionaries (posts) or None if an error occurs.
    """

    query = "SELECT * FROM reddit_posts WHERE subreddit = %s;"
    params = (subreddit,)
    posts = fetch_reddit_data(db_connection, query, params)
    return posts

In [9]:
def get_posts_by_subreddit(db_connection, subreddit, limit):  # Added limit parameter
    """Fetches posts from a specific subreddit with a limit.

    Args:
        db_connection: An instance of the DatabaseConnection class.
        subreddit: The name of the subreddit.
        limit: The maximum number of posts to retrieve (default is 10).

    Returns:
        A list of dictionaries (posts) or None if an error occurs.
    """

    query = "SELECT * FROM reddit_posts WHERE subreddit = %s LIMIT %s;" # Limit added to query
    params = (subreddit, limit) # Limit passed as a parameter
    posts = fetch_reddit_data(db_connection, query, params)
    return posts



# Example usage (with limit):

db_conn = DatabaseConnection()
if db_conn.connect():
    try:
        target_subreddit = "nonduality"  # Or any subreddit you want
        limit = 1 # Set your desired limit


        subreddit_posts = get_posts_by_subreddit(db_conn, target_subreddit, limit)


        if subreddit_posts:
            print(f"Posts from r/{target_subreddit} (limit {limit}):")
            for post in subreddit_posts:
                for field, value in post.items(): # Iterate through all fields/values
                    print(f"{field.capitalize()}: {value}") # Print field name and value
                print("-" * 20)  # Separator between posts
        else:
            print(f"No posts found for r/{target_subreddit} or an error occurred.")

    finally:
        db_conn.disconnect()

Successfully connected to MySQL database version 8.0.39
Posts from r/nonduality (limit 1):
Id: 1hflin4
Author: januszjt
Title: "If you begin to understand what you are, without trying to change it, then what you are undergoes a transformation." J Krishnamurti
Text: So, all effort must cease, it is futile you cannot be improved what you already are. Understand means to stand under and let illusions fall away.
Url: https://reddit.com/r/nonduality/comments/1hflin4/if_you_begin_to_understand_what_you_are_without/
Created_utc: 2024-12-16 15:25:02
Score: 7
Num_comments: 0
Subreddit: nonduality
Created_at: 2024-12-16 15:56:08
Updated_at: 2024-12-16 15:56:08
--------------------
Database connection closed


In [2]:
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv
from src.reddit import RedditScraper
from src.settings import Settings
from src.aws_handler import DatabaseConnection,get_posts_by_subreddit,fetch_reddit_data
#subreddit input
def get_list_from_input():
    while True:  # Loop until valid input is received
        try:
            input_str = input("Enter list elements separated by commas: ")
            my_list = [item.strip() for item in input_str.split(",")]
            return my_list  # Return the list if successful
        except ValueError: # This is usually not needed for split(",") but good practice if your separator might cause a ValueError with certain inputs.
            print("Invalid input. Please enter comma-separated values.")


sub_reddit = get_list_from_input()
post_limit=10
months=3
scraper=RedditScraper()
df=scraper.scrape_subreddit(subreddit_names=sub_reddit,post_limit=post_limit,months=months)
print(df)

db = DatabaseConnection()
connect=db.connect()
print(connect)


# add a button if append then do this 
append_data_to_db(db=db,df=df)
'''2nd part'''
# then to show the whole data do this 

db_conn = DatabaseConnection()
try:
    if db_conn.connect():
        limit = df.len()  # Set your desired limit

        query = "SELECT * FROM reddit_posts LIMIT %s;" # Limit in the query
        params = (limit,) # Limit passed as a parameter

        all_posts = fetch_reddit_data(db_conn, query, params)
        if all_posts:
            print(f"Retrieved {len(all_posts)} posts (limit was {limit}):")
            for post in all_posts:
                print(post)
        else:
             print("No posts found or an error occurred.") # Handle the case where no posts are found

finally:  # Ensure the connection is closed even if errors occur
    db_conn.disconnect()

#or show the data based upon the sub reddit
#show the all subreddit options 

db_conn = DatabaseConnection()
if db_conn.connect():
    try:
        target_subreddit = input("subreddit filter")  # Or any subreddit you want show the available options
        limit = 1 # Set your desired limit


        subreddit_posts = get_posts_by_subreddit(db_conn, target_subreddit, limit)


        if subreddit_posts:
            print(f"Posts from r/{target_subreddit} (limit {limit}):")
            for post in subreddit_posts:
                for field, value in post.items(): # Iterate through all fields/values
                    print(f"{field.capitalize()}: {value}") # Print field name and value
                print("-" * 20)  # Separator between posts
        else:
            print(f"No posts found for r/{target_subreddit} or an error occurred.")

    finally:
        db_conn.disconnect()

ModuleNotFoundError: No module named 'settings'

In [23]:
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv
import pandas as pd  # Import pandas
from src.reddit import RedditScraper
from src.settings import Settings
from src.aws_handler import DatabaseConnection

# ... (your existing code)

sub_reddit = get_list_from_input()
post_limit = 1
months = 3
scraper = RedditScraper()
df = scraper.scrape_subreddit(subreddit_names=sub_reddit, post_limit=post_limit, months=months)
print(df)

db = DatabaseConnection()
connect = db.connect()
print(connect)

# --- Append DataFrame to database ---

try:
    cursor = connect.cursor()

    # 1. Get column names from DataFrame
    columns = ", ".join(df.columns)

    # 2. Create placeholders for values
    placeholders = ", ".join(["%s"] * len(df.columns))

    # 3. SQL query to insert data (replace 'your_table_name' with your actual table name)
    sql = f"INSERT INTO your_table_name ({columns}) VALUES ({placeholders})"

    # 4. Convert DataFrame rows to tuples for insertion
    data = [tuple(row) for row in df.itertuples(index=False)] # index=False to exclude the index

    # 5. Execute the query with data
    cursor.executemany(sql, data)


    # 6. Commit the changes
    connect.commit() 
    print(f"{cursor.rowcount} rows inserted successfully into table")


except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if connect.is_connected():
        cursor.close()
        connect.close()

2024-12-16 21:13:59,933 - RedditScraper - INFO - Attempting to create Reddit client
2024-12-16 21:13:59,933 - RedditScraper - INFO - Attempting to create Reddit client
2024-12-16 21:13:59,933 - RedditScraper - INFO - Attempting to create Reddit client
2024-12-16 21:13:59,933 - RedditScraper - INFO - Attempting to create Reddit client
2024-12-16 21:13:59,933 - RedditScraper - INFO - Attempting to create Reddit client
2024-12-16 21:13:59,933 - RedditScraper - INFO - Attempting to create Reddit client
2024-12-16 21:13:59,933 - RedditScraper - INFO - Attempting to create Reddit client
2024-12-16 21:13:59,933 - RedditScraper - INFO - Attempting to create Reddit client
2024-12-16 21:13:59,939 - RedditScraper - INFO - Reddit client successfully created and verified
2024-12-16 21:13:59,939 - RedditScraper - INFO - Reddit client successfully created and verified
2024-12-16 21:13:59,939 - RedditScraper - INFO - Reddit client successfully created and verified
2024-12-16 21:13:59,939 - RedditScrap

        id                author                              title  \
0  1hfgjj2  Particular_Young3797  Python Subprocess BlockingIOError   

                                                text  \
0  Hi Python developers,\n\nAnyone know about the...   

                                                 url  \
0  https://reddit.com/r/Python/comments/1hfgjj2/p...   

                created_utc  score  num_comments subreddit  
0 2024-12-16 10:51:18+00:00      0             0    python  
Successfully connected to MySQL database version 8.0.39
True


AttributeError: 'bool' object has no attribute 'is_connected'

In [38]:

append_data_to_db(db=db,df=df)

Error inserting batch: 1062 (23000): Duplicate entry '1hfgjj2' for key 'reddit_posts.PRIMARY'
Successfully appended 1 records to reddit_posts


In [1]:
def append_data_to_db(db, df):
    """
    Append data to a database table
    
    Args:
        db: Database connection object
        df: Pandas DataFrame to append
        table_name: Name of the table to append to
    """
    try:
        # Prepare insert query
        columns = df.columns.tolist()
        placeholders = ', '.join(['%s'] * len(columns))
        column_names = ', '.join(columns)
        
        insert_query = f"""
        INSERT INTO {"reddit_posts"} ({column_names})
        VALUES ({placeholders})
        """
        
        # Convert DataFrame to list of tuples
        values = df.fillna('').to_records(index=False).tolist()
        
        # Batch insert
        batch_size = 1000
        for i in range(0, len(values), batch_size):
            batch = values[i:i+batch_size]
            
            try:
                db.cursor.executemany(insert_query, batch)
                db.connection.commit()
                print(f"Inserted batch from {i} to {i+len(batch)}")
            except Exception as batch_error:
                print(f"Error inserting batch: {batch_error}")
                db.connection.rollback()
        
        print(f"Successfully appended {len(values)} records to {"reddit_posts"}")
    
    except Exception as e:
        print(f"Error during append: {e}")

In [5]:
df.columns

Index(['id', 'author', 'title', 'text', 'url', 'created_utc', 'score',
       'num_comments', 'subreddit'],
      dtype='object')

In [15]:
import pandas as pd
append_data_to_db(db=db,df=df,table_name="reddit01")

Error inserting batch: 'NoneType' object has no attribute 'executemany'
Error during append: 'NoneType' object has no attribute 'rollback'


In [1]:
def get_list_from_input():
    while True:  # Loop until valid input is received
        try:
            input_str = input("Enter list elements separated by commas: ")
            my_list = [item.strip() for item in input_str.split(",")]
            return my_list  # Return the list if successful
        except ValueError: # This is usually not needed for split(",") but good practice if your separator might cause a ValueError with certain inputs.
            print("Invalid input. Please enter comma-separated values.")


my_list = get_list_from_input()
print(my_list)

['dog', 'cat']


In [6]:
from src.aws_handler import DatabaseConnection, import_scraped_data_to_db
from src.reddit import RedditScraper

# Create database connection
db = DatabaseConnection()

# Print the type and value of db to see what's happening
print("Database connection object:", db)
print("Type of db:", type(db))

# Check if the connection is valid
if db is None:
    print("Database connection failed!")
else:
    try:
        # Check if the connection has the necessary methods
        print("Connection methods:", dir(db))
    except Exception as e:
        print("Error accessing connection:", e)

Database connection object: <src.aws_handler.DatabaseConnection object at 0x000001F21439F4D0>
Type of db: <class 'src.aws_handler.DatabaseConnection'>
Connection methods: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'config', 'connect', 'connection', 'cursor', 'disconnect']


In [15]:
import os
from dotenv import load_dotenv
import mysql.connector
from mysql.connector import Error

# Load environment variables
load_dotenv()

# Debugging function to check database connection
def debug_database_connection():
    try:
        # Print out environment variables for debugging
        print("DB_HOST:", os.getenv('DB_HOST'))
        print("DB_NAME:", os.getenv('DB_NAME'))
        print("DB_USER:", os.getenv('DB_USER'))
        
        # Attempt to establish a direct connection
        connection = mysql.connector.connect(
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            port=os.getenv("port")
        )
        
        # Check if connection is successful
        if connection.is_connected():
            print("Database connection successful!")
            cursor = connection.cursor()
            print("Cursor created successfully")
            
            # Close connection
            cursor.close()
            connection.close()
            return True
        else:
            print("Failed to connect to the database")
            return False
    
    except Error as e:
        print(f"Error connecting to MySQL database: {e}")
        return False

# Run the diagnostic
debug_database_connection()

DB_HOST: redditdb.cbkuy486ce24.ap-south-1.rds.amazonaws.com
DB_NAME: reddit01
DB_USER: admin
Database connection successful!
Cursor created successfully


True